## Example soil moisture validation: ASCAT - ISMN

This example shows how to setup the pytesmo validation framework to perform a comparison between ASCAT and ISMN data. 

In [2]:
import os
import tempfile

import pytesmo.validation_framework.metric_calculators as metrics_calculators

from datetime import datetime

from ascat.timeseries import AscatSsmCdr
from pytesmo.io.ismn.interface import ISMN_Interface
from pytesmo.validation_framework.validation import Validation
from pytesmo.validation_framework.results_manager import netcdf_results_manager

/home/cpa/.pyenv/versions/miniconda-3.16.0/envs/pytesmo/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


First we initialize the data readers that we want to use. In this case the ASCAT soil moisture time series and in situ data from the ISMN. 

Initialize ASCAT reader

In [19]:
ascat_data_folder = os.path.join('/home', 'cpa', 'workspace', 'pytesmo',
                                 'tests', 'test-data', 'sat', 'ascat', 'netcdf', '55R22')
ascat_grid_folder = os.path.join('/media/sf_R', 'Datapool_processed', 'WARP',
                                 'ancillary', 'warp5_grid')
static_layers_folder = os.path.join('/home', 'cpa', 'workspace', 'pytesmo',
                                    'tests', 'test-data', 'sat',
                                    'h_saf', 'static_layer')


ascat_reader = AscatSsmCdr(ascat_data_folder, ascat_grid_folder,
                           static_layer_path=static_layers_folder)

Initialize ISMN reader

In [20]:
ismn_data_folder = '/data/Development/python/workspace/pytesmo/tests/test-data/ismn/multinetwork/header_values/'
ismn_reader = ISMN_Interface(ismn_data_folder)

The validation is run based on jobs. A job consists of at least three lists or numpy arrays specifing the grid point index, its latitude and longitude. In the case of the ISMN we can use the `dataset_ids` that identify every time series in the downloaded ISMN data as our grid point index. We can then get longitude and latitude from the metadata of the dataset.

**DO NOT CHANGE** the name ***jobs*** because it will be searched during the parallel processing!

In [21]:
jobs = []

ids = ismn_reader.get_dataset_ids(variable='soil moisture', min_depth=0, max_depth=0.1)
for idx in ids:
    metadata = ismn_reader.metadata[idx]
    jobs.append((idx, metadata['longitude'], metadata['latitude']))
print jobs

[(0, 102.13330000000001, 33.666600000000003), (1, 102.13330000000001, 33.883299999999998), (2, -120.9675, 38.430030000000002), (3, -120.78559, 38.149560000000001), (4, -120.80638999999999, 38.17353), (5, -105.417, 34.25), (6, -97.082999999999998, 37.133000000000003), (7, -86.549999999999997, 34.783000000000001)]


For this small test dataset it is only one job

It is important here that the ISMN reader has a read_ts function that works by just using the `dataset_id`. In this way the validation framework can go through the jobs and read the correct time series.

In [22]:
data = ismn_reader.read_ts(ids[0])
print data.head()

                     soil moisture soil moisture_flag soil moisture_orig_flag
date_time                                                                    
2008-07-01 00:00:00           0.45                  U                       M
2008-07-01 01:00:00           0.45                  U                       M
2008-07-01 02:00:00           0.45                  U                       M
2008-07-01 03:00:00           0.45                  U                       M
2008-07-01 04:00:00           0.45                  U                       M


## Initialize the Validation class

The Validation class is the heart of the validation framwork. It contains the information about which datasets to read using which arguments or keywords and if they are spatially compatible. It also contains the settings about which metric calculators to use and how to perform the scaling into the reference data space. It is initialized in the following way:

In [23]:
datasets = {'ISMN': {'class': ismn_reader, 
                     'columns': ['soil moisture']},
            'ASCAT': {'class': ascat_reader, 'columns': ['sm'],
                      'kwargs': {'mask_frozen_prob': 80,
                                 'mask_snow_prob': 80,
                                 'mask_ssf': True}}
           }

The datasets dictionary contains all the information about the datasets to read. The `class` is the dataset class to use which we have already initialized. The `columns` key describes which columns of the dataset interest us for validation. This a mandatory field telling the framework which other columns to ignore. In this case the columns `soil moisture_flag` and `soil moisture_orig_flag` will be ignored by the ISMN reader. We can also specify additional keywords that should be given to the `read_ts` method of the dataset reader. In this case we want the ASCAT reader to mask the ASCAT soil moisture using the included frozen and snow probabilities as well as the SSF. There are also other keys that can be used here. Please see the documentation for explanations.

In [24]:
period = [datetime(2007, 1, 1), datetime(2014, 12, 31)]
basic_metrics = metrics_calculators.BasicMetrics(other_name='k1')

process = Validation(
    datasets, 'ISMN', {(2, 2): basic_metrics.calc_metrics},
    temporal_ref='ASCAT',
    scaling='lin_cdf_match',
    scaling_ref='ASCAT',   
    period=period)


During the initialization of the Validation class we can also tell it other things that it needs to know. In this case it uses the datasets we have specified earlier. The spatial reference is the `'ISMN'` dataset which is the second argument. The third argument looks a little bit strange so let's look at it in more detail.

It is a dictionary with a tuple as the key and a function as the value. The key tuple `(n, k)` has the following meaning: `n` datasets are temporally matched together and then given in sets of `k` columns to the metric calculator. The metric calculator then gets a DataFrame with the columns ['ref', 'k1', 'k2' ...] and so on depending on the value of k. The value of `(2, 2)` makes sense here since we only have two datasets and all our metrics also take two inputs. 

This can be used in more complex scenarios to e.g. have three input datasets that are all temporally matched together and then combinations of two input datasets are given to one metric calculator while all three datasets are given to another metric calculator. This could look like this:

```python
{ (3 ,2): metric_calc,
  (3, 3): triple_collocation}
```

Create the variable ***save_path*** which is a string representing the path where the results will be saved. **DO NOT CHANGE** the name ***save_path*** because it will be searched during the parallel processing!

In [25]:
save_path = tempfile.mkdtemp()

In [26]:
import pprint
for job in jobs:
    
    results = process.calc(*job)
    pprint.pprint(results)
    netcdf_results_manager(results, save_path)


/home/cpa/.pyenv/versions/miniconda-3.16.0/envs/pytesmo/lib/python2.7/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/cpa/.pyenv/versions/miniconda-3.16.0/envs/pytesmo/lib/python2.7/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


{(('ASCAT', 'sm'), ('ISMN', 'soil moisture')): {'BIAS': array([-0.04330891], dtype=float32),
                                                'R': array([ 0.7128256], dtype=float32),
                                                'RMSD': array([ 7.72966719], dtype=float32),
                                                'gpi': array([0], dtype=int32),
                                                'lat': array([ 33.6666]),
                                                'lon': array([ 102.1333]),
                                                'n_obs': array([384], dtype=int32),
                                                'p_R': array([ 0.], dtype=float32),
                                                'p_rho': array([ 0.], dtype=float32),
                                                'p_tau': array([ nan], dtype=float32),
                                                'rho': array([ 0.70022893], dtype=float32),
                                                'tau': array([ 

The validation is then performed by looping over all the defined jobs and storing the results. 
You can see that the results are a dictionary where the key is a tuple defining the exact combination of datasets and columns that were used for the calculation of the metrics. The metrics itself are a dictionary of `metric-name: numpy.ndarray` which also include information about the gpi, lon and lat. Since all the information contained in the job is given to the metric calculator they can be stored in the results.

Storing of the results to disk is at the moment supported by the `netcdf_results_manager` which creates a netCDF file for each dataset combination and stores each metric as a variable. We can inspect the stored netCDF file which is named after the dictionary key:

In [27]:
import netCDF4
results_fname = os.path.join(save_path, 'ASCAT.sm_with_ISMN.soil moisture.nc')

with netCDF4.Dataset(results_fname) as ds:
    for var in ds.variables:
        print var, ds.variables[var][:]

n_obs [ 384  357  482  141  251 1927 1887 1652]
tau [ nan  nan  nan  nan  nan  nan  nan  nan]
gpi [0 1 2 3 4 5 6 7]
RMSD [  7.72966719  11.58347607  14.57700157  13.06224251  12.90389824
  14.24668026  21.19682884  17.3883934 ]
lon [ 102.1333   102.1333  -120.9675  -120.78559 -120.80639 -105.417    -97.083
  -86.55   ]
p_tau [ nan  nan  nan  nan  nan  nan  nan  nan]
BIAS [-0.04330891  0.237454   -0.63301021 -1.9682411  -0.21823417 -0.14228749
  0.2600247  -0.04437888]
p_rho [  0.00000000e+00   2.47165110e-28   0.00000000e+00   4.62621032e-39
   4.20389539e-45   3.32948515e-42   0.00000000e+00   0.00000000e+00]
rho [ 0.70022893  0.53934574  0.69356072  0.84189808  0.74206454  0.30299741
  0.53143877  0.62204134]
lat [ 33.6666   33.8833   38.43003  38.14956  38.17353  34.25     37.133
  34.783  ]
R [ 0.7128256   0.4996146   0.78071409  0.79960084  0.80635566  0.50703788
  0.53643185  0.6058206 ]
p_R [  0.00000000e+00   6.12721281e-24   0.00000000e+00   1.38538225e-32
   0.00000000e+00   

## Parallel processing

The same code can be executed in parallel by defining the following `start_processing` function.

In [28]:
def start_processing(job):
    try:
        return process.calc(*job)
    except RuntimeError:
        return process.calc(*job)

`pytesmo.validation_framework.start_validation` can then be used to run your validation in parallel. 
Your setup code can look like this Ipython notebook without the loop over the jobs. Otherwise the validation would be done twice. Save it into a `.py` file e.g. `my_validation.py`.

After [starting the ipyparallel cluster](http://ipyparallel.readthedocs.org/en/latest/process.html) you can then execute the following code:

```python
from pytesmo.validation_framework import start_validation

# Note that before starting the validation you must start a controller
# and engines, for example by using: ipcluster start -n 4
# This command will launch a controller and 4 engines on the local machine.
# Also, do not forget to change the setup_code path to your current setup.

setup_code = "my_validation.py"
start_validation(setup_code)
```

## Masking datasets

Masking datasets are datasets that return a pandas DataFrame with boolean values. `True` means that the observation should be masked, `False` means it should be kept. All masking datasets are temporally matched in pairs to the temporal reference dataset. Only observations for which all masking datasets have a value of `False` are kept for further validation.

The masking datasets have the same format as the dataset dictionary and can be specified in the Validation class with the `masking_datasets` keyword.

### Masking adapter

To easily transform an existing dataset into a masking dataset `pytesmo` offers a adapter class that calls the `read_ts` method of an existing dataset and performs the masking based on an operator and a given threshold.

In [29]:
from pytesmo.validation_framework.adapters import MaskingAdapter

ds_mask = MaskingAdapter(ismn_reader, '<', 0.2)
print ds_mask.read_ts(ids[0])['soil moisture'].head()

date_time
2008-07-01 00:00:00    False
2008-07-01 01:00:00    False
2008-07-01 02:00:00    False
2008-07-01 03:00:00    False
2008-07-01 04:00:00    False
Name: soil moisture, dtype: bool
